In [2]:
library(here)
library(here)
library(readr, warn.conflicts = FALSE)
library(arrow, warn.conflicts = FALSE)
library(magrittr, warn.conflicts = FALSE)
library(stringr, warn.conflicts = FALSE)
library(dplyr, warn.conflicts = FALSE)
library(rlang, warn.conflicts = FALSE)
library(data.table, warn.conflicts = FALSE)
library(lubridate, warn.conflicts = FALSE)
library(tidyr, warn.conflicts = FALSE)
library(truveta.study)
library(dbplyr)
library(lubridate)

here() starts at /home/jovyan/studyrepositories/hypertensioncohorts-is5fyzbqztbebenalfbuo4boem

Loading required package: DBI


Attaching package: ‘dbplyr’


The following objects are masked from ‘package:dplyr’:

    ident, sql




In [3]:
# build connections 
con <- create_connection()
study <- get_study(con, title = "Hypertension Cohort")
population <- get_population(con, study, title = "Study Cohort")
snapshot <- get_latest_snapshot(con, population, finished = TRUE)

[1] "Deprecation warning: the Vital and SocialHistory tables will be removed from the Truveta Data Model on September 12th, 2023. New snapshots generated after this date will exclude these tables. Existing snapshots which include these tables will be unaffected. Visit Learn for more information."


To sign in to Truveta, use a web browser to open the page https://login.truveta.com/activate?user_code=MNFQ-FFBT.

Signed in successfully.



In [4]:
# prepare concept table
concept <- read_parquet("data/Concept.parquet")
unidentified_conditions <- c(0,1067557,1067561)
params <- list(
    'codes'=unidentified_conditions
)

In [9]:
# basic descriptive data
sql <- "select count(distinct Id) as cnt from Condition;"

load_sql_table(con,snapshot,query = sql)%>%collect()

cnt
<int>
735047629


In [10]:
# basic descriptive data
sql <- "select count(distinct Id) as cnt from Condition
where RecordedDateTime is not NULL;"

load_sql_table(con,snapshot,query = sql)%>%collect()

cnt
<int>
609338994


In [8]:
sql <- "WITH t as (
SELECT PersonId, row_number() over (partition by PersonId order by RecordedDateTime),
RecordedDateTime, AbsoluteEffectiveDateTime 
FROM SearchResult_hypertension
GROUP BY PersonId, RecordedDateTime, AbsoluteEffectiveDateTime),
indexDT as (
SELECT
PersonId,
RecordedDateTime,
AbsoluteEffectiveDateTime,
DATEDIFF(DAY, RecordedDateTime, AbsoluteEffectiveDateTime + '-01') as ShiftDays
FROM t
WHERE rownum=1
ORDER BY PersonId);

SELECT count(c.*) as cnt
FROM Condition c
JOIN indexDT i ON (c.PersonId = i.PersonId)
WHERE
c.RecordedDateTime IS NOT NULL AND
year(c.RecordedDateTime + ShiftDays) = 2019
GROUP BY c.PersonId, c.RecordedDateTime;"


In [11]:
# find condition category distribution
sql <- "select s.PrimaryDiagnosisConceptId, c.ConceptName, count(s.Id) as Records
from Condition s join Concept c ON (s.PrimaryDiagnosisConceptId = c.ConceptId)
group by PrimaryDiagnosisConceptId, ConceptName
order by 1;"

primary_diagnosis_count <- load_sql_table(con,snapshot,query = sql)%>% collect()
primary_diagnosis_count

PrimaryDiagnosisConceptId,ConceptName,Records
<int>,<chr>,<int>
1067557,Field is not present in source,327712537
1067561,No Information,2123482
1200405,No,343797222
1200406,Yes,61414388


In [7]:
# find records's code system distribution of category over code concept

sql <- "select c.CategoryConceptId, cc.CodeConceptId, count(distinct c.Id) as RecordsCount, count(distinct c.PersonId) as PatientCount
from Condition c left join ConditionCodes cc on (c.Id = cc.ConditionId and c.PersonId = cc.PersonId)
group by CategoryConceptId, CodeConceptId
order by 1;"

concept <- read_parquet("data/Concept.parquet")
category_codeconcept <- load_sql_table(con,snapshot,query = sql)%>% collect()
cat_concept_name <- category_codeconcept %>% merge(.,concept,by.x = "CategoryConceptId", by.y = "ConceptId", all.x =TRUE)%>%
rename(CategoryName = ConceptName, CategoryCodeSystem = CodeSystem )%>% 
merge(.,concept,by.x = 'CodeConceptId',by.y='ConceptId', all.x = TRUE)%>%
rename(DiagnosisName=ConceptName,DiagnosisCodeSystem=CodeSystem)


,CodeConceptId,CategoryConceptId,RecordsCount,PatientCount,CategoryName,ConceptDefinition.x,CategoryCodeSystem,ConceptCode.x,ConceptClass.x,Domain.x,__index_level_0__.x,DiagnosisName,ConceptDefinition.y,DiagnosisCodeSystem,ConceptCode.y,ConceptClass.y,Domain.y,__index_level_0__.y
,<dbl>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,0,1065172,17,4,MedicalHistory,MedicalHistory,Truveta,1065172,ConditionCategory,Categories,264679,Field has not been mapped,The field has not been mapped.,NullFlavor,NM,NullFlavor,Categories,22724
2,0,1065168,140,125,AdmissionDiagnosis,AdmissionDiagnosis,Truveta,1065168,ConditionCategory,Categories,264675,Field has not been mapped,The field has not been mapped.,NullFlavor,NM,NullFlavor,Categories,22724
3,0,1065169,899,756,BillingDiagnosis,BillingDiagnosis,Truveta,1065169,ConditionCategory,Categories,264676,Field has not been mapped,The field has not been mapped.,NullFlavor,NM,NullFlavor,Categories,22724
4,10,1065170,2,2,EncounterDiagnosis,EncounterDiagnosis,Truveta,1065170,ConditionCategory,Categories,264677,Anxiety disorder of childhood OR adolescence,Anxiety disorder of childhood OR adolescence (disorder),SNOMED CT,109006,Clinical Finding,Condition,22734
5,10,1065174,1,1,ProblemList,ProblemList,Truveta,1065174,ConditionCategory,Categories,264680,Anxiety disorder of childhood OR adolescence,Anxiety disorder of childhood OR adolescence (disorder),SNOMED CT,109006,Clinical Finding,Condition,22734
6,10,1065172,23,1,MedicalHistory,MedicalHistory,Truveta,1065172,ConditionCategory,Categories,264679,Anxiety disorder of childhood OR adolescence,Anxiety disorder of childhood OR adolescence (disorder),SNOMED CT,109006,Clinical Finding,Condition,22734
7,11,1065172,52,3,MedicalHistory,MedicalHistory,Truveta,1065172,ConditionCategory,Categories,264679,Parathyroid structure,Parathyroid structure (body structure),SNOMED CT,111002,BodySiteExamined,Categories,22735
8,11,1065174,6,6,ProblemList,ProblemList,Truveta,1065174,ConditionCategory,Categories,264680,Parathyroid structure,Parathyroid structure (body structure),SNOMED CT,111002,BodySiteExamined,Categories,22735
9,11,1065168,1,1,AdmissionDiagnosis,AdmissionDiagnosis,Truveta,1065168,ConditionCategory,Categories,264675,Parathyroid structure,Parathyroid structure (body structure),SNOMED CT,111002,BodySiteExamined,Categories,22735


In [8]:
sum(category_codeconcept$RecordsCount)
sum(category_codeconcept$PatientCount)

[1] 1520893001

[1] 227376924

In [14]:
# join Encounter with Condition to find patients with encounter but no diagnosis

sql <- "select e.ClassConceptId as EncounterClassId, c.CategoryConceptId as ConditionCategory,
count(distinct e.Id) as ConditionCount, count(distinct e.PersonId) as PatientCount
from Encounter e left join Condition c on(e.Id = c.EncounterId)
where e.StatusConceptId not in (2983199,2506590,2983200,2506591,2506595)
and (year(e.StartDateTime) in (2017,2018))
group by e.ClassConceptId, c.CategoryConceptId
order by 4;"

ent_con <- load_sql_table(con,snapshot, query = sql)%>% collect()

In [16]:
sum(ent_con$ConditionCount)
sum(ent_con$PatientCount)

load_sql_table(con,snapshot,query = "select count(distinct Id) as cnt from Encounter e
where e.StatusConceptId not in (2983199,2506590,2983200,2506591,2506595)
and (year(e.StartDateTime) in (2017,2018)); ")%>% collect()

[1] 332584081

[1] 102510682

cnt
<int>
62030029


In [17]:
ent_con_data <- ent_con %>% collect()%>% merge(.,concept,by.x = "ConditionCategory", by.y = "ConceptId")%>%
rename(CategoryName = ConceptName, CategoryCodeSystem = CodeSystem )%>% 
merge(.,concept,by.x = 'DiagnosisId',by.y='ConceptId')%>%
rename(DiagnosisName=ConceptName,DiagnosisCodeSystem=CodeSystem)%>%
merge(.,concept, by.x = "EncounterClassId", by.y = "ConceptId") %>%
 rename(EncounterClaa = ConceptName, EncounterCodeSystem = CodeSystem)

In [18]:
head(ent_con_data,5)

,EncounterClassId,DiagnosisId,ConditionCategory,ConditionCount,PatientCount,CategoryName,ConceptDefinition.x,CategoryCodeSystem,ConceptCode.x,ConceptClass.x,⋯,ConceptClass.y,Domain.y,__index_level_0__.y,EncounterClaa,ConceptDefinition,EncounterCodeSystem,ConceptCode,ConceptClass,Domain,__index_level_0__
,<int>,<int>,<int>,<int>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,⋯,<chr>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,0,589599,1065170,1,1,EncounterDiagnosis,EncounterDiagnosis,Truveta,1065170,ConditionCategory,⋯,,Condition,194450,Field has not been mapped,The field has not been mapped.,NullFlavor,NM,NullFlavor,Categories,22724
2,0,76399,1065170,131,101,EncounterDiagnosis,EncounterDiagnosis,Truveta,1065170,ConditionCategory,⋯,Clinical Finding,Condition,369,Field has not been mapped,The field has not been mapped.,NullFlavor,NM,NullFlavor,Categories,22724
3,0,465510,1065170,8,4,EncounterDiagnosis,EncounterDiagnosis,Truveta,1065170,ConditionCategory,⋯,Clinical Finding,Condition,51463,Field has not been mapped,The field has not been mapped.,NullFlavor,NM,NullFlavor,Categories,22724
4,0,537800,1065168,3,3,AdmissionDiagnosis,AdmissionDiagnosis,Truveta,1065168,ConditionCategory,⋯,,Condition,141974,Field has not been mapped,The field has not been mapped.,NullFlavor,NM,NullFlavor,Categories,22724
5,0,536394,1065170,24,6,EncounterDiagnosis,EncounterDiagnosis,Truveta,1065170,ConditionCategory,⋯,,Condition,140563,Field has not been mapped,The field has not been mapped.,NullFlavor,NM,NullFlavor,Categories,22724


In [19]:
write_csv(ent_con_data,"data/encounter_condition.csv")

In [6]:
write_csv(cat_concept_name,"data/category_concept_conditions.csv")

In [9]:
sql <- "select cc.CodeConceptId, count( *) as cnt
from ConditionCodes cc join Condition c on (c.Id= cc.ConditionId and c.SourceProvenanceConceptId = cc.SourceConceptId)
group by cc.CodeConceptId;"

code_count <- load_sql_table(con,snapshot,query = sql)

In [10]:
sum(code_count$cnt)

Scalar
1772036021

In [13]:
# find patients who have more than one primary diagnosis condition record
sql <- "select top 100 *
from (select PatientId, count(Id) as Records
from Condition
where PrimaryDiagnosisConceptId = 1200406
and CategoryConceptId is not NULL and
CategoryConceptId not in :codes
group by PatientId) as t
where t.Records > 5;
"

patient_subset <- load_sql_table(con,snapshot,query=sql,params= params) %>%collect()


In [ ]:
#patient_subset

In [ ]:
# check if a 

In [1]:
#get a subset of the condition table with designated patient ids
patient_ids <- c("b1307ccf-a2fe-4d0f-4460-a16a781a24b8","e29dd1b1-bc5b-f79e-128a-d6cf9a69c6f0",
          "6aceabf2-ffa1-5d66-1fb7-29dc6645aae1" , "7529f9b9-d8c5-0d14-a968-302e78157141","c2d25038-d7e6-873c-5352-21232e2b4ce4",
              "1eb608b6-a393-cf31-2350-7ee3b12cdf85","cae257a0-c324-92f9-e0b9-d06b94b3f910", "c3846a8c-b925-a389-a392-3d20b7f90e6f",
               "4e0eecf1-f9ca-bcce-ae45-169461f3e0c1" )
params <- list(PatientId = patient_ids)

sql <- "select con.*, c.CodeConceptId
from Condition con join ConditionCodes c on (con.Id = c.ConditionId)
where con.PatientId in :PatientId;"

patient_sample <- load_sql_table(con,snapshot,query = sql, params = params)%>% collect()
head(patient_sample)

ERROR: Error in load_sql_table(con, snapshot, query = sql, params = params) %>% : could not find function "%>%"


In [ ]:
head(patient_sample)

In [6]:
write_csv(patient_sample,"data/condition_patients.csv")

In [7]:
# find patients who have more than one valid condition records
sql <- "select s.CodeConceptId,c.ConceptName, count(distinct s.ConditionId) as 'RecordCount'
from ConditionCodes s join Concept c on (s.CodeConceptId = c.ConceptId)
group by CodeConceptId, ConceptName
order by 1;"

condition_count <- load_sql_table(con,snapshot,query  = sql) %>%collect()
condition_count

CodeConceptId,ConceptName,RecordCount
<int>,<chr>,<int>
0,Field has not been mapped,8434254
10,Anxiety disorder of childhood OR adolescence,26
11,Parathyroid structure,60
22,Choroidal hemorrhage,8
38,No past history of,1319
40,Chronic pharyngitis,6289
41,Glaucoma as birth trauma,11
45,Colloid milium,6
61,Thermal injury,40


In [4]:
# check conditioncodes table's uniqueness

sql <- "
select top 50 * from
(select ConditionId, count(*) as total 
from ConditionCodes
group by ConditionId) as t
where t.total >=2
;"

cc <- load_sql_table(con,snapshot, query = sql)%>% collect()

In [5]:
params = list('ConditionId' = cc$ConditionId)
sql <- "select * from ConditionCodes
where ConditionId in :ConditionId;
"
example <- load_sql_table(con,snapshot, query = sql,params = params) %>% collect()

In [6]:
concept = read_parquet("data/Concept.parquet")
example_code <- example %>% merge(.,concept,by.x = "CodeConceptId", by.y = "ConceptId") 
head(example_code,10)

,CodeConceptId,ConditionId,SourceConceptId,PersonId,ConceptName,ConceptDefinition,CodeSystem,ConceptCode,ConceptClass,Domain,__index_level_0__
,<int>,<chr>,<int>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>,<int>
1,721,f2ff5f22-9fcb-e307-6067-9cfe6a2195d8,2703595,45ec6601-3d9a-6221-04ae-572d4517a50d,Actinic keratosis,Actinic keratosis (morphologic abnormality),SNOMED CT,856006,Morph Abnormality,Observation,18916
2,3429,e7f42f2f-c608-fb25-55ba-c7f8f42a288e,2703597,b6fd229d-5ca7-9ad2-61de-cb3cb91925b0,Arthritis,Arthritis (disorder),SNOMED CT,3723001,Clinical Finding,Condition,22711
3,3429,e7f42f2f-c608-fb25-55ba-c7f8f42a288e,2703595,b6fd229d-5ca7-9ad2-61de-cb3cb91925b0,Arthritis,Arthritis (disorder),SNOMED CT,3723001,Clinical Finding,Condition,22711
4,8125,cab7a762-b693-e7c3-5eb4-9304f4ec7a78,2703597,98ea710a-f8c5-8f96-c68e-20d34c90186c,Aortic valve disorder,Aortic valve disorder (disorder),SNOMED CT,8722008,Clinical Finding,Condition,30803
5,8125,cab7a762-b693-e7c3-5eb4-9304f4ec7a78,2703595,98ea710a-f8c5-8f96-c68e-20d34c90186c,Aortic valve disorder,Aortic valve disorder (disorder),SNOMED CT,8722008,Clinical Finding,Condition,30803
6,12365,6c18334b-1362-e657-9d10-58ed8c4ffa84,2703597,368fccf8-324d-1ac6-0477-500c7ee0daa9,Peptic ulcer,Peptic ulcer (disorder),SNOMED CT,13200003,Clinical Finding,Condition,37469
7,12780,654ddb07-bb59-0e3b-126a-5917e89fdba7,2703597,8c696c14-8eae-666e-b61d-5a3b1f9c381c,Hypercholesterolemia,Hypercholesterolemia (disorder),SNOMED CT,13644009,Clinical Finding,Condition,38986
8,12780,654ddb07-bb59-0e3b-126a-5917e89fdba7,2703595,8c696c14-8eae-666e-b61d-5a3b1f9c381c,Hypercholesterolemia,Hypercholesterolemia (disorder),SNOMED CT,13644009,Clinical Finding,Condition,38986
9,29813,79aa70b0-72ef-d171-99d1-f60326278333,2703597,68adb7a4-fca0-d857-ce57-93a469008631,Trigeminal neuralgia,Trigeminal neuralgia (disorder),SNOMED CT,31681005,Clinical Finding,Condition,65826


In [7]:
write_csv(example_code, "data/conditionCodeExample.csv.r")

In [18]:
sql <- "select ConditionId, PersonId, CodeConceptId, count(*) as total 
from ConditionCodes cc 
group by ConditionId,PersonId, CodeConceptId
order by 4 desc
;"

cc <- load_sql_table(con,snapshot, query = sql)%>% collect() %>% merge(., concept,by.x = 'CodeConceptId',by.y = 'ConceptId')


In [ ]:
head(cc,10)

In [10]:
# join encounter with condition to see if there will be duplicates.
sql <- "select count(e.Id) as cnt 
from Encounter e left join Condition c on (e.Id = c.EncounterId)
where e.StatusConceptId not in (2983199,2506590,2983200,2506591,2506595)
"

ent_con <- load_sql_table(con,snapshot,query = sql) %>% collect()
ent_con

cnt
<int>
755875327


In [11]:
sql <- "select count(distinct ConditionId) as cnt from ConditionCodes; "

load_sql_table(con, snapshot,query = sql)%>% collect()


cnt
<int>
726114247


In [5]:
# cut only source ending 94

sql <- "select cc.CodeConceptId as DiagnosisId, c.CategoryConceptId as ConditionCategory, count(*) as NRecords, count(distinct c.PersonId) as NPatient
from Condition c join ConditionCodes cc on (c.Id=cc.ConditionId and c.PersonId =cc.PersonId)
where SourceConceptId like '%94'
group by CodeConceptId, CategoryConceptId
order by 3;"

cc <- load_sql_table(con,snapshot,query = sql) %>% collect() %>%merge(.,concept,by.x = "ConditionCategory", by.y = "ConceptId")%>%
rename(CategoryName = ConceptName, CategoryCodeSystem = CodeSystem )%>% 
merge(.,concept,by.x = 'DiagnosisId',by.y='ConceptId')%>%
rename(DiagnosisName=ConceptName,DiagnosisCodeSystem=CodeSystem)


In [7]:
sum(cc$NRecords)/735047629 *100

[1] 94.86728

In [5]:
#count # of encounters in condition

sql <- "select count(distinct EncounterId) as total_ent 
from Condition;"


load_sql_table(con,snapshot,query = sql)%>% collect()

total_ent
<int>
110030147
